In [1]:
import tensorflow as tf
import numpy as np

CUDA_VISIBLE_DEVICES=3

/home/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/anaconda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

In [2]:
%env CUDA_VISIBLE_DEVICES= 3

env: CUDA_VISIBLE_DEVICES=3


In [3]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

In [6]:
num_dic, dic_len


({'a': 0,
  'b': 1,
  'c': 2,
  'd': 3,
  'e': 4,
  'f': 5,
  'g': 6,
  'h': 7,
  'i': 8,
  'j': 9,
  'k': 10,
  'l': 11,
  'm': 12,
  'n': 13,
  'o': 14,
  'p': 15,
  'q': 16,
  'r': 17,
  's': 18,
  't': 19,
  'u': 20,
  'v': 21,
  'w': 22,
  'x': 23,
  'y': 24,
  'z': 25},
 26)

In [17]:
seq_data = ['word', 'wood', 'deep' ,'dive', 'cold',
            'cool', 'load', 'love', 'kiss', 'kind']

In [18]:
def make_batch(seq_data):
    input_batch, target_batch = [], []
    
    for seq in seq_data: #학습 data에서 element하나씩 빼온다. work, wood
        input = [num_dic[n] for n in seq[:-1]] #work면 wor하나씩 들어오고 인덱스 값을 딕셔너리통해 참조
        target = num_dic[seq[-1]] #마지막단어 word면 d만 가져오게 모든 단어를 인덱스
        input_batch.append(np.eye(dic_len)[input]) #onehotvector np.eye <-단위벡터 1이면 [[1,0,0],[0,1,0]] 중에 0,1,0가져옴
        target_batch.append(target) #index값자체로.. api특성 타겟자체는 그냥 인덱스...
        
    return input_batch, target_batch

In [19]:
learning_rate = 0.01
n_hidden = 128 #rnn의 hidden사이즈
total_epoch = 30 #전체 몇번 학습
n_step = 3 #3개의 인풋을 넣고 하나의 아웃풋
n_input = n_class = dic_len #charector몇개인지 정의

In [55]:
X = tf.placeholder(tf.float32, [None, n_step, n_input]) #input. onehotvector 3*n_input
Y = tf.placeholder(tf.int32, [None]) #target



<tf.Tensor 'Placeholder_9:0' shape=(?,) dtype=int32>

In [60]:
W = tf.Variable(tf.random_normal([n_hidden, n_class])) #하나의 layer가 더 있다. 히든사이즈 벡터(27로 디멘션해주는). target에 대한 확률분포로 만듬
b = tf.Variable(tf.random_normal([n_class])) #letter수

<tf.Variable 'Variable_13:0' shape=(26,) dtype=float32_ref>

In [22]:
#w o r받고 wb를 통해서 프로젝션 - 최종 27차원 벡터 생성 -> d에 해당하는 것이 가장 크도록
#cell하나지만 2개로도 할수있음

In [62]:
# Fill in the details
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1,output_keep_prob = 0.5) #성능을 높이는 기능
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

In [27]:
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1,cell2])
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
#각셀의 히든값을 가져온다

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [28]:
#마지막 히드값만 가져온다.
outputs = tf.transpose(outputs, [1,0,2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

In [29]:
#optimazation 지정
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [37]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch,target_batch = make_batch(seq_data)
                                     
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={X: input_batch, 
                                 Y: target_batch})
    print('Epoch:',"%04d" % (epoch+1), 'cost = ', '{:.5f}' .format(loss))

    

Epoch: 0001 cost =  4.02940
Epoch: 0002 cost =  3.03183
Epoch: 0003 cost =  1.67495
Epoch: 0004 cost =  1.67385
Epoch: 0005 cost =  0.94271
Epoch: 0006 cost =  0.81004
Epoch: 0007 cost =  0.66810
Epoch: 0008 cost =  0.49524
Epoch: 0009 cost =  0.39608
Epoch: 0010 cost =  0.54629
Epoch: 0011 cost =  0.22243
Epoch: 0012 cost =  0.14859
Epoch: 0013 cost =  0.17424
Epoch: 0014 cost =  0.56028
Epoch: 0015 cost =  0.31271
Epoch: 0016 cost =  0.09686
Epoch: 0017 cost =  0.09754
Epoch: 0018 cost =  0.22883
Epoch: 0019 cost =  0.23809
Epoch: 0020 cost =  0.11166
Epoch: 0021 cost =  0.03796
Epoch: 0022 cost =  0.12981
Epoch: 0023 cost =  0.04382
Epoch: 0024 cost =  0.22438
Epoch: 0025 cost =  0.01477
Epoch: 0026 cost =  0.02609
Epoch: 0027 cost =  0.01949
Epoch: 0028 cost =  0.01037
Epoch: 0029 cost =  0.04118
Epoch: 0030 cost =  0.01847


In [49]:
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
prediction_check = tf.equal(prediction, Y)
accuracy= tf.reduce_mean(tf.cast(prediction_check, tf.float32))

In [50]:
input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run(
    [prediction, accuracy], feed_dict={X: input_batch, Y : target_batch})

predict_words = []
for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char) #input과 모델의 케릭터 합쳐서 단어를 만듬pur

In [52]:
print('Inserted:', [w[:] + ' ' for w in seq_data])
print('Predicted: ', predict_words)
print('Accuracy:', accuracy_val)

Inserted: ['word ', 'wood ', 'deep ', 'dive ', 'cold ', 'cool ', 'load ', 'love ', 'kiss ', 'kind ']
Predicted:  ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
Accuracy: 1.0
